In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from io import StringIO
import sys
import requests
from bs4 import BeautifulSoup
import pickle
import os
import glob
from pathlib import Path

In [289]:
shs_df = pickle.load(open("data/covers.p", "rb"))

In [192]:
shs_df.head()

,trackID,artistID,shsPerf,clique_id,name,title,year
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,-1,0,Louis Armstrong,Stardust,1988.0
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,-1,0,Artie Shaw and his orchestra,Stardust,1988.0
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,-1,0,Hoagy Carmichael,Star Dust,1999.0
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,-1,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,NaN
4,TROJZTF128F428B546,ARJN76O1187FB43C99,-1,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,2001.0


In [4]:
#artists_id = pd.read_csv("data/AdditionalFiles/unique_artists.txt")
artists_id = pd.read_csv("~/Desktop/unique_artists.txt",delimiter='<SEP>',engine='python',header=None,index_col=1,names=['artistID','artistMID','randomTrack','name'])

In [128]:
artist = "? & The Mysterians"
# Handle some special cases
if artist == "98º":
    artist = "98 degrees"
if artist == "4hero":
    artist = "4 hero"
if artist == "? & The Mysterians":
    artist = "the mysterians"
r = requests.get('https://secondhandsongs.com/search?search_text='+format_string(artist))
soup = BeautifulSoup(r.text, 'html.parser')
if "Search" in (soup.find("head")).find("title").text:
    table = soup.find('section', id="section-artist")
    title = table.find('h2', id='t1')
    # Check if the result is an artist
    if "Artists" in title.text:
    # Get the artist ID to find the its web page on SHS
        # When only one artist is found
        if "1+" in title.text or title.text == "Artists 1":
            artist_id = str(table.a).split("\"")[1]
        # When several artists are found
        else:
            artists = table.find_all("a")
            for art in artists:
                if art.text != "\n\n":
                    if format_string(artist) == format_string(art.text):
                        artist_id = str(art).split("\"")[1]
                        break

        r_artist = requests.get('https://secondhandsongs.com'+artist_id)
        soup = BeautifulSoup(r_artist.text, 'html.parser')
    else:
        print(title.text)
        print("Unavailable")
try:
    country = soup.find("dd", itemprop="nationality").text
    print(country)
except:
    country = "Unavailable"
    print(country)
    

United States


Create dataframe of unique artists with their location

In [6]:
artists = list(shs_df['name'])
unique_artists = pd.DataFrame(list(np.unique(artists)), columns = ["name"])
unique_artists['id'] = unique_artists['name'].map\
(lambda x : shs_df.get_value(shs_df[shs_df['name'] == x].head(1).index[0], 'artistID'))

In [198]:
import unidecode
def format_string(x):
    return unidecode.unidecode(x).lower().replace("&", "and").replace("_", " ").replace("'", "").replace(".", "")

In [139]:
get_artist_location("10_000 Maniacs")

'United States'

In [275]:
def get_artist_location(artist):
    artist = artist
    # Handle some special cases
    if artist == "98º":
        artist = "98 degrees"
    if artist == "4hero":
        artist = "4 hero"
    if artist == "? & The Mysterians":
        artist = "the mysterians"
    if artist == "crosby_ stills":
        artist = "crosby stills"
    r = requests.get('https://secondhandsongs.com/search?search_text='+format_string(artist))
    soup = BeautifulSoup(r.text, 'html.parser')
    if "Search" in (soup.find("head")).find("title").text:
        table = soup.find('section', id="section-artist")
        try :
            title = table.find('h2', id='t1')
        except:
            print("=============================================")
            print("ERROR 1 : no section artist for => " + artist)
            print("=============================================") 
            return "Unavailable"
        # Check if the result is an artist
        if "Artists" in title.text:
        # Get the artist ID to find the its web page on SHS
            # When only one artist is found
            if "1+" in title.text or title.text == "Artists 1":
                if "Mysterians" in artist:
                    print("mysterians 1+")
                artist_id = str(table.a).split("\"")[1]
            # When several artists are found
            else:
                if "Mysterians" in artist:
                    print("mysterians bug number results")
                artists = table.find_all("a")
                artist_id = ""
                for art in artists:
                    if art.text != "\n\n":
                        if format_string(artist) in format_string(art.text):
                            artist_id = str(art).split("\"")[1]
                            break
                if artist_id == "":
                    print("=============================================")
                    print("ERROR 2 : no artist_id for => "+ artist)
                    print("=============================================")
                    return "Unavailable"
    
            r_artist = requests.get('https://secondhandsongs.com'+artist_id)
            soup = BeautifulSoup(r_artist.text, 'html.parser')
        else:
            print(title.text)
            print("Unavailable")
            return "Unavailable"
    try:
        country = soup.find("dd", itemprop="nationality").text
        return country
    except:
        country = "Missing"
        return country

In [187]:
#unique_artists["country"] = unique_artists["name"].map(lambda x : get_artist_location(x))
#pickle.dump(unique_artists, open("data/first_location.p", "wb"))
unique_artists = pickle.load(open("data/first_location.p", "rb"))


In [206]:
unknown_loc = unique_artists[unique_artists['country'] == "Unavailable"].reset_index()
unknown_loc.head(10)

,index,name,country
0,1,1 Giant Leap feat. Michael Stipe & Asha Bhosle,Unavailable
1,17,50 Cent ft. Young Buck,Unavailable
2,29,A*Teens,Unavailable
3,33,A3,Unavailable
4,50,Ace Karaoke Productions,Unavailable
5,63,Adema,Unavailable
6,94,Alabama 3 / Tenor Fly,Unavailable
7,113,Alexandra Burke Feat. Ne-Yo,Unavailable
8,125,Alison Krauss / Union Station,Unavailable
9,126,Alison Limerick,Unavailable


By looking at the data for which we did not find a country, we observe that for a lot of them this is due to the fact that they are not single artists but featurings. We then decide to assign them the country of the first artist in the featuring. For those cases, we establish a list of separators that represent a featuring (e.g. "feat", "and", or "/"). Then we keep only the first artist in the list, and we proceed as before to find its country.

In [281]:
def location_featuring(artist):
    sep_list = ["with", ";", " ft. ", "feat", " and ", "/", "&"]
    sep = ""
    i = 0
    country = ""
    while (sep == "") and i < len(sep_list):
        if sep_list[i] in artist.lower():
            sep = sep_list[i]
            artist = artist.lower().split(sep)[0].replace("_", "")
            if artist[-1] == " ":
                artist = artist[:-1]
            country = get_artist_location(artist)
            break
        i = i+1
    if country == "":
        return "Unavailable"
    else:
        return country
                    
    

In [ ]:
feat_loc = unknown_loc[['name']]
feat_loc['country'] = feat_loc['name'].map(lambda x : location_featuring(x))

In [ ]:
feat_loc2 = feat_loc[feat_loc['country'] == "Unavailable"][['name']]
feat_loc2['country'] = feat_loc2['name'].map(lambda x : location_featuring(x))

In [ ]:
unique_artists["country"] = unique_artists["name"].map(lambda x : get_artist_location(x))

In [285]:
for n in feat_loc2['name']:
    i = unique_artists[unique_artists['name'] == n].index[0]
    if unique_artists.get_value(i, 'country') == "Unavailable":
        unique_artists.set_value(i, 'country', feat_loc2.get_value(feat_loc2[feat_loc2['name'] == n].index[0], 'country'))

In [287]:
len(unique_artists[unique_artists['country'] == "Unavailable"])

201

In [290]:
shs_df['country'] = shs_df['name'].map(lambda x : unique_artists.get_value(unique_artists[unique_artists['name'] == x].index[0], 'country'))

In [291]:
shs_df[shs_df['country'] == "Unavailable"]

,trackID,artistID,shsPerf,clique_id,name,title,year,country
13,TRLLCAL128F428B903,ARP2RHS1187B991595,-1,5,Zen Guerrilla,Moonage Daydream,NaN,Unavailable
18,TRYEKWE128F1459C42,AR6AK6B1187FB48EDD,-1,8,The Soggy Bottom Boys / Dan Tyminski,I Am A Man Of Constant Sorrow,2002.0,Unavailable
21,TRTTEXX128F428BE1D,ARJVH1S1187FB378C3,-1,9,JFA,Lowrider,2003.0,Unavailable
76,TRPKERG128F92CA8F1,AR5US1D1187B9AEB1F,-1,35,Jay Selway,Insecure Me,1982.0,Unavailable
134,TRJVQBD128F427D05B,ARFK4281187FB5C811,-1,62,Vox Dei,Genesis,1971.0,Unavailable
207,TRMWABD128F425FC39,ARS1OWB1187B99EEAD,-1,94,The New York Dolls,Don't Start Me Talking,1981.0,Unavailable
214,TRSQZFC128F92D28F5,AR6XT331187B9A349F,-1,97,Tracy Gang Pussy,Don't Drag Me Down,2005.0,Unavailable
239,TRGIWZA128F930936E,ARX56SC1187B9BA075,-1,109,Seru Giran,Canción De Alicia En El País,1980.0,Unavailable
240,TRTTKLD128F9340FB5,ARYKSA71187FB43186,-1,109,Fabiana Cantilo,Cancion De Alicia En El Pais,2005.0,Unavailable
255,TRTAPSC128F9300821,ARMLRNE1187FB59A6A,-1,117,Jimmy Rodgers,The Brakeman's Blues,NaN,Unavailable


In [292]:
#pickle.dump(shs_df, open("data/covers_loc.p", "wb"))

In [293]:
#pickle.dump(unique_artists, open("data/artists_location.p", "wb"))